# UAV Path Planning with Metaheuristic A*

In [80]:
## Defining G,H and F functions

#G cost = distance from starting node
#H cost = distance from end node
#F cost = G cost + H cost 
#we choose the node with least F cost , when F is equal we check H 

## Defining Input, Output

#INPUT : OccupancyGrid(matrix with 0:free 1:occupied), start(state : x,y), goal(x,y)
#OUTPUT : path (sequence of states (x,y,orientation))

## Pseudo Algo

In [81]:
'''
List OPEN: nodes to be evaluated 
List CLOSED : nodes already evaluted 
add the start node to OPEN
loop
  current = node is OPEN with the lowest f_cost
  remove current from OPEN
  add current to CLOSED
  if current is the target node
    return
for each neighbour of the current node
    if neighbour is not traversable or neighbour is in CLOSED
        skip to the next neighbour
    if new path to neighbor is shorter or neighbour is not in OPEN
        set f_cost to neighbour
        set parent of neighbour to current
        if neighbour is not in OPEN
            add neighbour to OPEN
'''

# ORIENTATION IS NOT INCLUDED (CHECK IT USING NEIGHBOUR)

'\nList OPEN: nodes to be evaluated \nList CLOSED : nodes already evaluted \nadd the start node to OPEN\nloop\n  current = node is OPEN with the lowest f_cost\n  remove current from OPEN\n  add current to CLOSED\n  if current is the target node\n    return\nfor each neighbour of the current node\n    if neighbour is not traversable or neighbour is in CLOSED\n        skip to the next neighbour\n    if new path to neighbor is shorter or neighbour is not in OPEN\n        set f_cost to neighbour\n        set parent of neighbour to current\n        if neighbour is not in OPEN\n            add neighbour to OPEN\n'

In [82]:
# ImageToMatrix
'''Function that transform an image to a matrix '''

'Function that transform an image to a matrix '

In [83]:
# Neighbours(node):
'''Function that returns the Neighbours of a node'''

'Function that returns the Neighbours of a node'

## A Star

In [84]:
'''
OccupancyGrid  n*m matrix (with 0,1)
moving up,down,right,left: cost is 1
moving rightUp,rightDown,leftUp,leftDown: cost is 1,4 (sqrt(2))
'''

'\nOccupancyGrid  n*m matrix (with 0,1)\nmoving up,down,right,left: cost is 1\nmoving rightUp,rightDown,leftUp,leftDown: cost is 1,4 (sqrt(2))\n'

In [1]:
import pygame
import math
from queue import PriorityQueue
#Window 
WIDTH = 900
WIN = pygame.display.set_mode((WIDTH,WIDTH))

pygame.display.set_caption("A* Path Finding Algorithm")

#Colors
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
PURPLE = (128, 0, 128)
ORANGE= (255, 165, 0)
GREY = (128, 128, 128)
TURQUOISE = (64, 224, 208)

pygame 2.1.2 (SDL 2.0.16, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
class Cell:
    def __init__(self,row,col,width,total_rows):
        self.row= row
        self.col=col
        self.x=row * width
        self.y=col * width
        self.color=WHITE
        self.neighbors = []
        self.width=width
        self.total_rows=total_rows

    def get_position(self):
        return self.row, self.col

    def is_closed(self):
        return self.color == RED

    def is_open(self):
        return self.color == GREEN

    def is_obstacle(self):
        return self.color == BLACK

    def is_start(self):
        return self.color == ORANGE

    def is_end(self):
        return self.color == TURQUOISE

    
    def reset(self):
        self.color = WHITE

    def make_close(self):
        self.color = RED

    def make_start(self):
        self.color = ORANGE

    def make_open(self):
        self.color = GREEN

    def make_obstacle(self):
        self.color = BLACK
    
    def make_end(self):
        self.color = TURQUOISE

    def make_path(self):
        self.color = PURPLE
   
    def draw(self,win):
        pygame.draw.rect(win,self.color,(self.x,self.y,self.width,self.width))

    def update_neighbors(self,grid):
        pass
    def __lt__(self,other):
        return False

In [4]:
 #Win functions

def h(p1,p2):
    x1, y1 =p1
    x2, y2 =p2
    return abs(x1-x2)+abs(y1-y2)

def make_grid(rows,width):
    grid =[]
    gap = width // rows #width of a single cell
    for i in range(rows):
        grid.append([])
        for j in range(rows):
            cell = Cell(i,j,gap,rows)
            grid[i].append(cell)
    return grid

def draw_grid(win,rows,width):
    gap = width // rows
    for i in range(rows):
        pygame.draw.line(win,GREY,(0,i*gap),(width,i*gap))
        for j in range(rows):
            pygame.draw.line(win,GREY,(j*gap,0),(j*gap,width))
def draw(win,grid,rows,width):
    win.fill(WHITE)
    for row in grid:
        for cell in row:
            cell.draw(win)
    draw_grid(win,rows,width)  
    pygame.display.update()


def get_clicked_pos(pos,rows,width):
    gap = width//rows
    y,x=pos
    
    row = y // gap
    col = x // gap

    return row,col     


In [5]:
def launch_win(win,width):
    ROWS = 50
    grid = make_grid(ROWS,width)

    start = None
    end = None
    run = True
    started = False
    while run:
        draw(win,grid,ROWS,width)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
            if started:
                continue

            if pygame.mouse.get_pressed()[0]: #LEFT
                pos = pygame.mouse.get_pos() 
                row,col=get_clicked_pos(pos,ROWS,width)
                cell = grid[row][col]
                if not start and cell != end:
                    start = cell
                    start.make_start()
                elif not end and cell != start:
                    end=cell
                    end.make_end()
                elif cell != end and cell != start:
                    cell.make_obstacle()

            elif pygame.mouse.get_pressed()[2]:
                pos = pygame.mouse.get_pos() 
                row,col=get_clicked_pos(pos,ROWS,width)
                cell = grid[row][col]
                cell.reset()
                if cell == start:
                    start =end
                if cell == end:
                    end = None
                
    pygame.quit()   


In [6]:

def main():
    launch_win(WIN,WIDTH)



In [91]:
def A_Star(occupancyGrid, start, goal):
    OPEN=[]   #nodes to be evaluated 
    CLOSED=[] #List of nodes already evaluted 
    PATH = [] #List of states (selected nodes) (state = {x,y,orientation})
    OPEN=start #start, goal are (x,y) on map 
    found=False
    while(not found):
        #current = (x,y) while  min(f_costs of neighbours)
        

##PS TRY TO INCLUDE ORIENTATION



SyntaxError: unexpected EOF while parsing (57580412.py, line 12)

: 

In [7]:
main()